In [12]:
import pandas as pd
import seaborn as sb
from gtfparse import read_gtf

In [255]:
ensmbl.head()

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,ccds_id,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version
0,1,ensembl_havana,gene,1211340,1214153,NaN,-,0,ENSG00000186827,11,...,,,,,,,,,,
1,1,ensembl_havana,transcript,1211340,1214153,NaN,-,0,ENSG00000186827,11,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS11,1,,,,,
2,1,ensembl_havana,exon,1213983,1214153,NaN,-,0,ENSG00000186827,11,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS11,1,1,ENSE00001832731,2,,
3,1,ensembl_havana,CDS,1213983,1214127,NaN,-,0,ENSG00000186827,11,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS11,1,1,,,ENSP00000368538,3
4,1,ensembl_havana,start_codon,1214125,1214127,NaN,-,0,ENSG00000186827,11,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS11,1,1,,,,


In [ ]:
from io import StringIO

s = StringIO()
x = [['1', 'havana', 'gene', '365389', '522928', '.', '-', '.', 'gene_id "ENSG00000237094"; gene_version "12"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene";'], ['1', 'havana', 'transcript', '501588', '517252', '.', '-', '.', 'gene_id "ENSG00000237094"; gene_version "12"; transcript_id "ENST00000641303"; transcript_version "1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; transcript_source "havana"; transcript_biotype "processed_transcript";'], ['1', 'havana', 'transcript', '501604', '517225', '.', '-', '.', 'gene_id "ENSG00000237094"; gene_version "12"; transcript_id "ENST00000641063"; transcript_version "1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; transcript_source "havana"; transcript_biotype "processed_transcript";'], ['1', 'havana', 'transcript', '504865', '522928', '.', '-', '.', 'gene_id "ENSG00000237094"; gene_version "12"; transcript_id "ENST00000642124"; transcript_version "1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; transcript_source "havana"; transcript_biotype "processed_transcript"; tag "basic";'], ['1', 'havana', 'exon', '517156', '517225', '.', '-', '.', 'gene_id "ENSG00000237094"; gene_version "12"; transcript_id "ENST00000641063"; transcript_version "1"; exon_number "1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudogene"; transcript_source "havana"; transcript_biotype "processed_transcript"; exon_id "ENSE00003812919"; exon_version "1";']]


In [115]:
ucsc = pd.read_csv("grch38.knownGene.txt.gz", sep='\t')

In [13]:
ensmbl = read_gtf("Homo_sapiens.GRCh38.105.chr.gtf.gz")

/Users/a233287/pve/lib/python3.8/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
/Users/a233287/pve/lib/python3.8/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']


In [2]:
gene_name_table = pd.read_csv("gene_name_table.txt", sep="\t")
medgenes = pd.read_csv("MRG_medgenes_names.txt")

In [27]:
len(medgenes)

10111

In [42]:
x = medgenes["gene"].value_counts()
redund = x[x > 1]
print(len(redund), "Redundant genes")

4981 Redundant genes


In [44]:
medgenes = medgenes.sort_values(['gene']).drop_duplicates()
len(medgenes)

5130

In [65]:
medgenes["has_table_query"] = medgenes["gene"].isin(gene_name_table["query"]).astype(bool)
medgenes["has_table_hit"] = medgenes["gene"].isin(gene_name_table[~gene_name_table["entrezgene"].isna()]["query"]).astype(bool)

In [66]:
medgenes.groupby(["has_table_query", "has_table_hit"]).size().unstack()

has_table_hit,False,True
has_table_query,,
False,17.0,NaN
True,44.0,5069.0


In [160]:
medgenes.loc[medgenes["has_ensmbl_hit"], 'ensmbl_name'] = medgenes.loc[medgenes["has_ensmbl_hit"]]['gene']

In [167]:
view = ~medgenes["has_ensmbl_hit"] & medgenes['symbol_has_ensmbl_hit']
medgenes.loc[view, 'ensmbl_name'] = medgenes.loc[view]['symbol']

In [174]:
missing = medgenes[medgenes['ensmbl_name'].isna()]
missing

,gene,has_table_query,has_table_hit,has_ensmbl_hit,symbol,symbol_has_ensmbl_hit,has_ucsc_geneName,symbol_has_ucsc_geneName,ensmbl_name
2521,C10orf11,False,False,False,NaN,NaN,False,False,NaN
2579,C10orf2,False,False,False,NaN,NaN,False,False,NaN
8261,C12orf65,False,False,False,NaN,NaN,True,False,NaN
493,C2orf71,False,False,False,NaN,NaN,False,False,NaN
1205,C4orf26,False,False,False,NaN,NaN,False,False,NaN
1351,C5orf42,False,False,False,NaN,NaN,False,False,NaN
7241,C8orf37,False,False,False,NaN,NaN,True,False,NaN
8986,CCL3L1,True,True,False,CCL3L1,False,True,True,NaN
3939,CCL4L1,True,True,False,CCL4L1,False,False,False,NaN
4665,GSTT1,True,True,False,GSTT1,False,True,True,NaN


In [217]:
len(missing)

14

In [218]:
#medgene - ensmbl_id - ensmbl_name - Notes
# C10orf11 - ENSG00000148655 - LRMDA
# C10orf2 - ENSG00000107815 - TWNK
# C12orf65 - ENSG00000130921 - MTRFR
# C2orf71 - ENSG00000179270 - PCARE
# C4orf26 - ENSG00000174792 - ODAPH
# C5orf42 - ENSG00000197603 - CPLANE1
# C8orf37 - ENSG00000156172 - CFAP418
# CCL3L1 -  ENSG00000276085 - CCL3L3
# CCL4L1 - Missing - - Might be ENSG00000276070, CCL4L2
# GSTT1 - Missing - - Closest ENSG00000277656
# PRSS3P2 - Missing - - Closest ENSG00000282667
# T - Missing - - No clue.. ENSG00000166349 / RAG1??
# TAS2R45 - Missing - - Closest ENSG00000261936
# TCEB3C - Missing - - Closest ENSG00000275553 / ELOA3CP
manual_translate = {"C10orf11" :"LRMDA",
"C10orf2" :"TWNK",
"C12orf65" :"MTRFR",
"C2orf71" :"PCARE",
"C4orf26" :"ODAPH",
"C5orf42" :"CPLANE1",
"C8orf37" :"CFAP418",
"CCL3L1" :"CCL3L3"}

In [220]:
medgenes.loc[medgenes['ensmbl_name'].isna(), 'ensmbl_name'] = medgenes.loc[medgenes['ensmbl_name'].isna(), 'gene'].map(manual_translate)

In [221]:
missing = medgenes[medgenes['ensmbl_name'].isna()]
missing

,gene,has_table_query,has_table_hit,has_ensmbl_hit,symbol,symbol_has_ensmbl_hit,has_ucsc_geneName,symbol_has_ucsc_geneName,ensmbl_name
3939,CCL4L1,True,True,False,CCL4L1,False,False,False,NaN
4665,GSTT1,True,True,False,GSTT1,False,True,True,NaN
2028,PRSS3P2,True,True,False,PRSS3P2,False,True,True,NaN
1800,T,False,False,False,NaN,NaN,False,False,NaN
8049,TAS2R45,True,True,False,TAS2R45,False,True,True,NaN
4150,TCEB3C,True,True,False,ELOA3BP,False,False,False,NaN


In [223]:
len(medgenes), len(medgenes[~medgenes["ensmbl_name"].isna()])

(5130, 5124)

In [228]:
pd.Series(medgenes["ensmbl_name"].unique(), name="ensmbl_name").to_csv("medgene_ensmbl_names.txt", index=False)

In [229]:
medgenes["ensmbl_name"].head()

7853      A1CF
2992       A2M
4732    A4GALT
1016     A4GNT
8135      AAAS
Name: ensmbl_name, dtype: object

In [234]:
ensmbl['feature'].unique()
# | -------  gene  -------- |
#    | --- transcript --- |
#

array(['gene', 'transcript', 'exon', 'CDS', 'start_codon', 'stop_codon',
       'five_prime_utr', 'three_prime_utr', 'Selenocysteine'],
      dtype=object)

In [245]:
ensmbl[(ensmbl["gene_name"] == "A4GNT") & (ensmbl['feature'] == 'gene')]

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,ccds_id,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version
581521,3,ensembl_havana,gene,138123713,138132390,NaN,-,0,ENSG00000118017,4,...,,,,,,,,,,


In [246]:
ensmbl[(ensmbl["gene_name"] == "A4GNT") & (ensmbl['feature'] == 'transcript')]

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,ccds_id,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version
581522,3,ensembl_havana,transcript,138123713,138132390,NaN,-,0,ENSG00000118017,4,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS3097,1,,,,,


In [248]:
ensmbl[(ensmbl["gene_name"] == "A4GNT") & (ensmbl['feature'] == 'exon')]

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,ccds_id,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version
581523,3,ensembl_havana,exon,138132212,138132390,NaN,-,0,ENSG00000118017,4,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS3097,1,1,ENSE00001241635,3,,
581524,3,ensembl_havana,exon,138130849,138131282,NaN,-,0,ENSG00000118017,4,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS3097,1,2,ENSE00001241655,1,,
581527,3,ensembl_havana,exon,138123713,138124878,NaN,-,0,ENSG00000118017,4,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS3097,1,3,ENSE00001037446,6,,


In [253]:
ensmbl.columns

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'gene_version', 'gene_name', 'gene_source',
       'gene_biotype', 'transcript_id', 'transcript_version',
       'transcript_name', 'transcript_source', 'transcript_biotype', 'tag',
       'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id',
       'exon_version', 'protein_id', 'protein_version'],
      dtype='object')

In [244]:
ensmbl[ensmbl['gene_name'].isin(medgenes['ensmbl_name'])].groupby(['gene_name', 'feature']).size().unstack()

feature,CDS,Selenocysteine,exon,five_prime_utr,gene,start_codon,stop_codon,three_prime_utr,transcript
gene_name,,,,,,,,,
A1CF,81.0,NaN,111.0,26.0,1.0,8.0,7.0,7.0,10.0
A2M,47.0,NaN,87.0,4.0,1.0,3.0,3.0,9.0,13.0
A4GALT,4.0,NaN,17.0,10.0,1.0,4.0,4.0,4.0,7.0
A4GNT,2.0,NaN,3.0,2.0,1.0,1.0,1.0,1.0,1.0
AAAS,59.0,NaN,240.0,13.0,1.0,4.0,4.0,5.0,22.0
...,...,...,...,...,...,...,...,...,...
ZNF81,14.0,NaN,22.0,10.0,1.0,4.0,3.0,3.0,5.0
ZNRF3,25.0,NaN,26.0,3.0,1.0,3.0,3.0,3.0,3.0
ZPBP,20.0,NaN,31.0,6.0,1.0,4.0,3.0,4.0,6.0


In [233]:
medgenes[medgenes["ensmbl_name"] == 'A2M']

,gene,has_table_query,has_table_hit,has_ensmbl_hit,symbol,symbol_has_ensmbl_hit,has_ucsc_geneName,symbol_has_ucsc_geneName,ensmbl_name
2992,A2M,True,True,True,A2M,True,True,True,A2M


In [165]:
medgenes[~medgenes["has_ensmbl_hit"]]['symbol_has_ensmbl_hit'].value_counts()

True     104
False      6
Name: symbol_has_ensmbl_hit, dtype: int64

In [142]:
medgenes["has_ensmbl_hit"] = medgenes['gene'].isin(ensmbl["gene_name"])
medgenes['has_ensmbl_hit'].value_counts()

True     5012
False     118
Name: has_ensmbl_hit, dtype: int64

In [67]:
medgenes["has_ensmbl_hit"] = medgenes[medgenes["has_table_hit"]]['gene'].isin(ensmbl["gene_name"])

In [ ]:
# ensmbl_gene_name = medgenes[medgenes["has_table_query"]]

In [70]:
medgenes[(medgenes['has_ensmbl_hit'] == False) & (medgenes['has_table_hit'] == True)]

,gene,has_table_query,has_table_hit,has_ensmbl_hit
3779,AARS,True,True,False
422,ADCK3,True,True,False
4672,ADRBK2,True,True,False
2349,AKAP2,True,True,False
4758,ARSE,True,True,False
...,...,...,...,...
4646,UFD1L,True,True,False
1894,WBSCR17,True,True,False
1118,WHSC1,True,True,False
1729,WISP3,True,True,False


In [86]:
gene_name_table['alias'] = gene_name_table['alias'].astype(str).apply(lambda x: [_ for _ in x.split(', ') if _ != 'nan'])

In [94]:
tmp = gene_name_table.set_index('query')
medgenes['symbol'] = medgenes['gene'].map(tmp['symbol'])

In [97]:
(medgenes['gene'] == medgenes['symbol'])

0.9732943469785575

In [98]:
medgenes["symbol_has_ensmbl_hit"] = medgenes[medgenes["has_table_hit"]]['symbol'].isin(ensmbl["gene_name"])

In [100]:
medgenes.groupby(['has_ensmbl_hit', 'symbol_has_ensmbl_hit']).size().unstack()

symbol_has_ensmbl_hit,False,True
has_ensmbl_hit,,
False,6,104
True,1,4958


In [122]:
# has ucsc
medgenes['has_ucsc_geneName'] = medgenes['gene'].isin(ucsc["geneName"])
medgenes['symbol_has_ucsc_geneName'] = medgenes['symbol'].isin(ucsc['geneName'])

In [125]:
missing = medgenes[(medgenes["has_ensmbl_hit"] == False) & (medgenes['symbol_has_ensmbl_hit'] == False) &
                  (medgenes["has_ucsc_geneName"] == False) & (medgenes['symbol_has_ucsc_geneName'] == False)]
missing

,gene,has_table_query,has_table_hit,has_ensmbl_hit,symbol,symbol_has_ensmbl_hit,has_ucsc_geneName,symbol_has_ucsc_geneName
3939,CCL4L1,True,True,False,CCL4L1,False,False,False
4150,TCEB3C,True,True,False,ELOA3BP,False,False,False


In [133]:
for _ in ["CCL4L1", "TCEB3C"]:
    print(_, _ in ucsc['geneName'], _ in ensmbl["gene_name"])
    for alias in gene_name_table[gene_name_table["query"] == _]['alias'].iloc[0]:
        print(alias, alias in ucsc['geneName'], alias in ensmbl["gene_name"])
    print()

CCL4L1 False False
AT744.2 False False
CCL4L False False
LAG-1 False False
LAG1 False False
MIP-1-beta False False

TCEB3C False False
ELOA3 False False
ELOA3B False False
ELOA3L1 False False
EloA3C False False
Elongin-A3 False False



In [134]:
ensmbl_manual_lookup = {"CCL4L1": "ENSG00000276070",
                        "TCEB3C":"ENSG00000288607"}

okay - now I need to go through the 5 pieces of keys and try to pull them each out.
Actually, wait - I need to take the *has_ucsc_geneName and try to map them to Ensmbl so I have a unified coordinate system

In [139]:
ucsc.columns

Index(['#chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand',
       'thickStart', 'thickEnd', 'reserved', 'blockCount', 'blockSizes',
       'chromStarts', 'name2', 'cdsStartStat', 'cdsEndStat', 'exonFrames',
       'type', 'geneName', 'geneName2', 'geneType', 'transcriptClass',
       'source', 'transcriptType', 'tag', 'level', 'tier'],
      dtype='object')

In [141]:
view = medgenes[(medgenes["has_ensmbl_hit"] == False) & (medgenes['symbol_has_ensmbl_hit'] == False) &
                   (medgenes["has_ucsc_geneName"] == True)]
ucsc[ucsc["geneName"].isin(view["gene"])]

,#chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,reserved,blockCount,...,type,geneName,geneName2,geneType,transcriptClass,source,transcriptType,tag,level,tier
196412,chr17,36194868,36196747,ENST00000612839.1,0,-,36194868,36194868,16646144,2,...,none,CCL3L1,none,none,nonCoding,havana_homo_sapiens,retained_intron,retained_intron_CDS,2,all
196413,chr17,36194868,36196758,ENST00000619989.1,0,-,36195285,36196673,789624,3,...,none,CCL3L1,P16619,none,coding,ensembl_havana_transcript_homo_sapiens,protein_coding,"CCDS,Ensembl_canonical,MANE_Select,appris_prin...",2,"canonical,basic,all"
196414,chr17,36194995,36196457,ENST00000616702.1,0,-,36194995,36194995,25600,3,...,none,CCL3L1,none,none,nonCoding,havana_homo_sapiens,processed_transcript,none,2,all
248196,chr7_KI270803v1_alt,773585,777110,ENST00000634192.1,0,+,773585,773585,16724991,5,...,none,PRSS3P2,none,none,pseudo,havana_homo_sapiens,unprocessed_pseudogene,"Ensembl_canonical,basic",2,"canonical,basic,all"
250652,chr12_GL877876v1_alt,327494,328424,ENST00000571573.1,0,-,327494,327494,16724991,1,...,none,TAS2R45,none,none,pseudo,havana_homo_sapiens,unprocessed_pseudogene,"Ensembl_canonical,basic,overlapping_locus",2,"canonical,basic,all"
257953,chr22_KI270879v1_alt,270313,278434,ENST00000628171.2,0,-,271169,278406,789624,5,...,none,GSTT1,A0A0G2JQM0,none,coding,havana_homo_sapiens,nonsense_mediated_decay,none,2,all
257954,chr22_KI270879v1_alt,270313,278442,ENST00000621513.3,0,-,273663,278406,789624,4,...,none,GSTT1,A0A0G2JMS2,none,coding,havana_homo_sapiens,nonsense_mediated_decay,none,2,all
257955,chr22_KI270879v1_alt,270313,278459,ENST00000612885.3,0,-,270597,278406,789624,5,...,none,GSTT1,P30711,none,coding,ensembl_havana_transcript_homo_sapiens,protein_coding,"Ensembl_canonical,basic",2,"canonical,basic,all"
257956,chr22_KI270879v1_alt,270498,278440,ENST00000629554.1,0,-,271147,278406,789624,3,...,none,GSTT1,A0A0G2JQD8,none,coding,havana_homo_sapiens,nonsense_mediated_decay,none,2,all
257957,chr22_KI270879v1_alt,270567,278421,ENST00000628969.2,0,-,270597,278406,789624,5,...,none,GSTT1,A0A0G2JRQ5,none,coding,havana_homo_sapiens,protein_coding,basic,2,"basic,all"


In [136]:
medgenes[(medgenes["has_ensmbl_hit"] == False) & (medgenes['symbol_has_ensmbl_hit'] == False) &
                  (medgenes["has_ucsc_geneName"] == False) & (medgenes['symbol_has_ucsc_geneName'] == True)]


,gene,has_table_query,has_table_hit,has_ensmbl_hit,symbol,symbol_has_ensmbl_hit,has_ucsc_geneName,symbol_has_ucsc_geneName
